In [30]:
# Imports
import pyfabricops as pf
import os
from pathlib import Path

# Setups
pf.set_auth_provider('env')
pf.setup_logging(format_style='minimal', level='info') 

In [31]:
# Find root path
def find_root_path(
    start: Path = None,
    markers: list[str] = None
) -> Path:
    start = (start or Path.cwd()).resolve()
    markers = markers or [
        ".git",         
        ".gitignore",
    ]
    for directory in [start, *start.parents]:
        for m in markers:
            if (directory / m).exists():
                return directory
    return start

root_path = find_root_path()
print(f"Root path: {root_path}")

Root path: C:\repos\pyfabricops-demo-002-fabric


In [32]:
# Capture git info
branches_path = Path(root_path) / 'branches.json'
branch = pf.get_current_branch()
workspace_suffix = pf.get_workspace_suffix(branch, path = branches_path) 
print('Branch:', branch, 'Workspace Suffix:', workspace_suffix) 

Branch: dev Workspace Suffix: -DEV


In [41]:
# Project config
project = 'PF_002_Fabric'
project_path = Path(root_path)/'src'
workspace_alias ='PF_002_Fabric'
workspace_name = workspace_alias + workspace_suffix
capacity = '7732a1eb-3893-4642-a85c-93fc3f35d076' 
print('Workspace name: ', workspace_name) 

Workspace name:  PF_002_Fabric-DEV


In [37]:
workspaces_roles_path = Path(root_path) / 'workspaces_roles.json'
roles = pf.read_json(workspaces_roles_path) 
display(pf.json_to_df(roles))  

,user_uuid,user_type,role
0,cbe83b10-3b7c-4479-8168-e5281fabf7ea,Group,Admin
1,9322eb4a-4132-4bd1-8df1-5cd3d1d2400b,User,Admin
2,b025341a-965a-4c35-b12e-2af63e5eb58f,User,Contributor
3,bde43861-55e1-4144-b572-be115312967f,ServicePrincipal,Admin


In [38]:
# Create the workspace

pf.create_workspace(
    workspace_name,
    description='CI/CD with Microsoft Fabric and PyFabricOps',
    capacity=capacity,
    roles=roles,
    df=True,
)

✓ Workspace PF_002_Fabric-DEV created successfully.
✓ User "cbe83b10-3b7c-4479-8168-e5281fabf7ea", type "Group" with role "Admin" was added to workspace 0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8 successfully.
✓ User "9322eb4a-4132-4bd1-8df1-5cd3d1d2400b", type "User" with role "Admin" was added to workspace 0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8 successfully.
✓ User "b025341a-965a-4c35-b12e-2af63e5eb58f", type "User" with role "Contributor" was added to workspace 0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8 successfully.
△ 409: {"requestId":"9d5c7ede-acd4-4666-ad55-4d588c8e606b","errorCode":"PrincipalAlreadyHasWorkspaceRolePermissions","moreDetails":[{"relatedResource":{"resourceId":"bde43861-55e1-4144-b572-be115312967f","resourceType":"Principal"}}],"message":"The provided principal already has a role assigned in the workspace","relatedResource":{"resourceId":"0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8","resourceType":"Workspace"}}.


,id,displayName,description,type,capacityId,capacityRegion,oneLakeEndpoints_blobEndpoint,oneLakeEndpoints_dfsEndpoint,capacityAssignmentProgress
0,0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8,PF_002_Fabric-DEV,CI/CD with Microsoft Fabric and PyFabricOps,Workspace,7732a1eb-3893-4642-a85c-93fc3f35d076,East US,https://eastus-onelake.blob.fabric.microsoft.com,https://eastus-onelake.dfs.fabric.microsoft.com,Completed


In [42]:
# Create project
pf.export_workspace_config(
    workspace_name,
    project_path,
    branches_path=branches_path,
    workspace_suffix=workspace_suffix
) 

△ No existing config found at C:\repos\pyfabricops-demo-002-fabric\src\config.json, creating a new one.
✓ Updated workspace_config for "PF_002_Fabric" in branch "dev"
✓ Workspace configuration successfully written to C:\repos\pyfabricops-demo-002-fabric\src\config.json


In [50]:
# Create `storage` folder
storage_folder = pf.create_folder(
    workspace_name,
    'Storage',
)
storage_folder_id = storage_folder['id'] 
display(pf.json_to_df(storage_folder))
print('Folder `storage` created with id: ', storage_folder_id) 

,id,displayName,workspaceId
0,4617f283-43b9-498a-96bc-3078391adcfb,Storage,0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8


Folder `storage` created with id:  4617f283-43b9-498a-96bc-3078391adcfb


In [51]:
# Create `Orchestration` folder
orchestration_folder = pf.create_folder(
    workspace_name,
    'Orchestration',
)
orchestration_folder_id = orchestration_folder['id'] 
display(pf.json_to_df(orchestration_folder))
print('Folder `Orchestration` created with id: ', orchestration_folder_id) 

,id,displayName,workspaceId
0,1813457d-633a-46bc-bd65-7afb6a563195,Orchestration,0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8


Folder `Orchestration` created with id:  1813457d-633a-46bc-bd65-7afb6a563195


In [52]:
# Create `PowerBI` folder
powerbi_folder = pf.create_folder(
    workspace_name,
    'PowerBI',
)
powerbi_folder_id = powerbi_folder['id'] 
display(pf.json_to_df(powerbi_folder))
print('Folder `PowerBI` created with id: ', powerbi_folder_id) 

,id,displayName,workspaceId
0,704abe3c-4c90-4a82-b5e9-55b8d046a452,PowerBI,0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8


Folder `PowerBI` created with id:  704abe3c-4c90-4a82-b5e9-55b8d046a452


In [53]:
# Create subfolders `DirectLake` and `Import` 
subfolders = ['DirectLake', 'Import']
parent_folder_id = powerbi_folder_id 
for subfolder in subfolders:
    subfolder_created = pf.create_folder(
        workspace_name, 
        subfolder, 
        parent_folder=powerbi_folder_id,
    )
    subfolder_created_id = subfolder_created['id']
    print(f'Folder `{subfolder}` created with id: {subfolder_created_id} under `PowerBI`')  


Folder `DirectLake` created with id: e38f6560-e5e8-4884-b227-c3e185d5249b under `PowerBI`
Folder `Import` created with id: 337bb477-27a9-4871-974f-65d2d116ee88 under `PowerBI`


In [54]:
# Export folders to config
pf.export_folders(
    workspace_name,
    project_path,
    branches_path=branches_path,
    workspace_suffix=workspace_suffix
) 

✓ Folder structure created at C:/repos/pyfabricops-demo-002-fabric/src/PF_002_Fabric
✓ Updated folders config for workspace "PF_002_Fabric" in branch "dev"
✓ Folders configuration successfully written to C:\repos\pyfabricops-demo-002-fabric\src\config.json.


In [ ]:
# Load config
config_path = Path(project_path)/'config.json'
config = pf.read_json(config_path)

{'dev': {'PF_002_Fabric': {'workspace_config': {'workspace_id': '0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8', 'workspace_name': 'PF_002_Fabric-DEV', 'workspace_description': 'CI/CD with Microsoft Fabric and PyFabricOps', 'capacity_id': '7732a1eb-3893-4642-a85c-93fc3f35d076', 'capacity_region': 'East US', 'workspace_roles': [{'user_uuid': 'cbe83b10-3b7c-4479-8168-e5281fabf7ea', 'user_type': 'Group', 'role': 'Admin', 'display_name': 'AppFabricUsers', 'group_type': 'SecurityGroup', 'email': None}, {'user_uuid': '9322eb4a-4132-4bd1-8df1-5cd3d1d2400b', 'user_type': 'User', 'role': 'Admin', 'display_name': 'Alison Pezzott', 'user_principal_name': 'alison.pezzott@overdax.com'}, {'user_uuid': 'b025341a-965a-4c35-b12e-2af63e5eb58f', 'user_type': 'User', 'role': 'Contributor', 'display_name': 'Dev', 'user_principal_name': 'dev@overdax.com'}, {'user_uuid': 'bde43861-55e1-4144-b572-be115312967f', 'user_type': 'ServicePrincipal', 'role': 'Admin', 'display_name': 'AppFabric', 'app_id': '7f3c52e1-46fa-4d5d

In [ ]:
workspace_id = config[branch][workspace_alias]['workspace_config']['workspace_id']
storage_folder_id_c = config[branch][workspace_alias]['folders']['Storage'] 

In [58]:
# Create lakehouse
pf.create_lakehouse(
    workspace_id,
    'MainStorage', 
    description='The main storage of the lakehouse',  
    folder=storage_folder_id_c,
    df=True
)

△ No valid lakehouses found in workspace '0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8'.


,id,type,displayName,description,workspaceId,folderId
0,78c4f663-1bda-4d19-9827-1e16eea65b6c,Lakehouse,MainStorage,The main storage of the lakehouse,0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8,4617f283-43b9-498a-96bc-3078391adcfb


In [60]:
# Export lakehouse to config
pf.export_lakehouse(
    workspace_id,
    'MainStorage',
    project_path,
    workspace_path=workspace_alias,
    branches_path=branches_path,
    workspace_suffix=workspace_suffix,
)

i Found existing config file at C:\repos\pyfabricops-demo-002-fabric\src\config.json, merging workspace config...


In [61]:
# Create warehouse
pf.create_warehouse(
    workspace_id,
    'MainWarehouse', 
    description='The SQL instance for the project',  
    folder=storage_folder_id_c,
    df=True
)

△ No valid warehouses found in workspace '0ae3ae7c-e2a9-407d-a92b-0bdc8799b9a8'.


In [63]:
# Export warehouse to config
pf.export_warehouse(
    workspace_id,
    'MainWarehouse',
    project_path,
    workspace_path=workspace_alias,
    branches_path=branches_path,
    workspace_suffix=workspace_suffix,
)

i Found existing config file at C:\repos\pyfabricops-demo-002-fabric\src\config.json, merging workspace config...
